In [9]:
# Este código é uma versão orientada a objetos do algoritmo de mineração de dados APRIORI
# Identifica padrões, gera candidatoes e constrói regras de associação
# implementada e idealizada por Rafael Stoffalette João.

# find me on https://github.com/rafaelstojoao/apyori

import numpy

class rule:
    def __init__(self):
        self.support    = 0.0
        self.confidence = 0.0
        self.lift       = 0.0
        self.X          = []
        self.Y          = []
        self.conviction = 0.0
        self.allItems   = []
        self.dataset    = []

    def printRule(self):
        print(self.X,'-->',self.Y,'sup: ',self.getSupport(),' conf:',self.getConfidence(), 'lift: ',self.lift, 'conviction: ',self.conviction)
        fpRules = open("./rules.rul","a+")
        fpRules.write(str(self.X)+'-->'+str(self.Y)+'sup: '+str(self.getSupport())+' conf:'+str(self.getConfidence())+ 'lift: '+str(self.lift)+ 'conviction: '+str(self.conviction)+"\n")

    def getConfidence(self):
        return self.confidence

    def getSupport(self):
        return self.support

    def setConfidence(self,conf):
        self.confidence = conf

    def setSupport(self,sup):
        self.support = sup



class itemset :
    def __init__(self):
        self.support    = 0.0
        self.counting   = 0
        self.items      = []

    def getitems(self):
        return self.items


    def setSupport(self,dataSetSize):
        self.suporte = self.counting / dataSetSize

    def getSupport(self):
        return self.suporte

    def getCounting(self):
        return self.counting

    def setCounting(self,count):
        self.counting = count

    def increaseCounting(self):
        self.counting =+ 1


class Apryori:

    def __init__(self):
        self.minSup     = 0.0
        self.minConf    = 0.0
        self.arq        = ""
        self.dataset    = []
        self.listadePadroes     = []
        self.listadeCandidatos  = []
        self.listRules          = []

    def loadDataSetFromFile(self, path):
        import csv
        data =[]
        with open(path) as csvfile:
            data = list(csv.reader(csvfile))
            self.dataset = sorted(data)

        print(len(self.dataset))
        print(self.dataset)
        return self.dataset


    def loadDataSet(self):
       print('Dataset Loaded')
       return [['pão','muçarela','presunto'],
               ['pão','muçarela','presunto','mortadela', 'leite'],
               ['pão','muçarela','mortadela'],
               ['pão','presunto','mortadela','leite'],
               ['pão','muçarela','presunto','mortadela'],
               ['mortadela','pão'],
               ['ovo','leite']]
        # return [['leite','pão'],['pão','manteiga'],['cerveja'],['leite','pão','manteiga'],['pão']  ]

    def combinaItens(self,elem,listaCombinar):
        combinados =[]
        print('combinar')
        print(elem)
        print('com')
        print(listaCombinar)
        for item in listaCombinar:
            combinados.append([elem,item])
        return combinados


    def aprioriGen(self,i):
        if(i == 0):
            print('generating Cand1')
            itemsTam1 = []
            for transaction in self.dataset:
                for item in transaction:
                     if item not in itemsTam1:
                        itemsTam1.append(item)
            itemsTam1.sort()
            listaCandidatostemp = []
            for item in itemsTam1: #é preciso transformar cada item em um candidato.
                cand  = itemset()
                cand.items = [item]
                listaCandidatostemp.append(cand)
            self.listadeCandidatos.append(listaCandidatostemp)

        elif len(self.listadePadroes[i - 1]) <= 1:
            print('Unable to build more candidates...')
            self.listadeCandidatos.append([])
            return
        else:
            print('\n\n generating Cand_' + str(i + 1))

            listaPadroes =[]
            listaCandidatos = []
            for padrao in self.listadePadroes[i-1]:
                listaPadroes.append(padrao.items)

            for i in range(len(listaPadroes)):
                padrao = listaPadroes[i]
                resto = listaPadroes[i+1:]

                for itset in resto:
                    L1 = padrao[:-1]
                    L2 = itset[:-1]

                    if L1 == L2:
                        candidato = sorted(list(set(padrao) | set(itset)))
                        # if(len(candidato) >= 10):
                        #     print(candidato)
                        cand = itemset()
                        cand.items =candidato
                        listaCandidatos.append(cand)
            self.listadeCandidatos.append(listaCandidatos)
            return



    def validaCandidatos(self,i):
        print('Validating candidates of size: ' + str(i+1))
        print('minsup: '+str(self.minSup))
        fp = open("./patterns.patt", "a+")
        if i==0: # se são itens unitários para verificar
            for reg in self.dataset:
                for cand in self.listadeCandidatos[i]:
                    if set(cand.items).intersection(reg):
                         cand.counting += 1
            listaPadroes = []

            for cand in self.listadeCandidatos[i]:
                cand.suporte = cand.counting/self.tamanhoDataset
                if(cand.suporte >= self.minSup):
                    listaPadroes.append(cand)
                    fp.write(str(cand.items) + ' support: ' + str(cand.suporte) + "\n")

            return listaPadroes

        else:
            for reg in self.dataset:
                for cand in self.listadeCandidatos[i]:

                    if self.sublist(cand.items,reg):
                        cand.counting+=1

            listaPadroes = []
            for cand in self.listadeCandidatos[i]:
                cand.suporte = cand.counting / self.tamanhoDataset
                if (cand.suporte >= self.minSup):
                    listaPadroes.append(cand)
                    fp.write(str(cand.items) + ' support: ' + str(cand.suporte) + "\n")
            return listaPadroes

    def sublist(self, lst1, lst2):
        condBool = True
        for elem in lst1:
            if elem not in lst2:
                condBool = False
        return condBool

    

    def estimateSupport(self,listitems):
        count =0
        for reg in self.dataset:
            if (set(listitems).intersection(reg) == set(listitems)):
                count += 1
        return count/self.tamanhoDataset

    

    def   apriori(self,dataset,minsup=None,minconf=None):
        self.dataset = dataset
        self.tamanhoDataset = len(self.dataset)
        self.minSup  = minsup
        self.minConf = minconf

        i = 0
        self.aprioriGen(0)
        print('Candidates of size: '+str(i+1))
        for itemsetCandidato in self.listadeCandidatos[0]:
              print(itemsetCandidato.items)


        while (self.listadeCandidatos[i]):
            self.listadePadroes.append(self.validaCandidatos(i))

            print('Patterns: ')
            # if(len(self.listadePadroes[i][1].items) >= 8):
            fp = open("patterns.csv","a+")

            for padrao in self.listadePadroes[i]:
                print(str(padrao.items)+' Support: '+str(padrao.suporte))
                fp.write(str(padrao.items)+' Support: '+str(padrao.suporte)+"\n")
            i += 1
            self.aprioriGen(i)
            for cand in self.listadeCandidatos[i]:
                print(cand.items)

#         self.generateRules()

#         for regra in self.listRules:
#             regra.printRule()



if __name__ == '__main__':
    apr = Apryori()
    print("pelase, insert a valid file path or -1 to run a simple example!")
    op = str(input())
    if(op == '-1'):
        print("ignoring data file, this example will run under dataset below:")
        dataset = apr.loadDataSet()
        print(dataset)
    else:
        print("This execution will run under "+ op+ "file")
        dataset = apr.loadDataSetFromFile(op)

    print("Set the minimun support parameter: (recomended 0.025)")
    minsup = float(input())
    
    apr.apriori(dataset,minsup,minconf)

pelase, insert a valid file path or -1 to run a simple example!
sales.csv
This execution will run under sales.csvfile
441
[['1', '1', '10379', '17972', '4748', '4686', '1547', '3265'], ['1', '1', '11092', '5008', '5249', '453', '392', '373'], ['1', '1', '11210', '3576', '5119', '561', '1682', '2398'], ['1', '1', '11686', '2154', '6824', '3527', '592', '697'], ['1', '1', '12680', '3243', '4157', '660', '761', '786'], ['1', '1', '13569', '346', '489', '2077', '44', '659'], ['1', '1', '14755', '899', '1382', '1765', '56', '749'], ['1', '1', '15218', '258', '1138', '2516', '333', '204'], ['1', '1', '1537', '3748', '5838', '1859', '3381', '806'], ['1', '1', '15603', '2703', '3833', '4260', '325', '2563'], ['1', '1', '15671', '5279', '2406', '559', '562', '572'], ['1', '1', '18044', '1475', '2046', '2532', '130', '1158'], ['1', '1', '18226', '659', '2914', '3752', '586', '578'], ['1', '1', '1838', '6380', '2824', '1218', '1216', '295'], ['1', '1', '18567', '1895', '1393', '1801', '244', '210

0.005
generating Cand1
Candidates of size: 1
['1']
['10']
['100']
['1000']
['10002']
['10044']
['10069']
['1009']
['10099']
['101']
['1012']
['1014']
['10155']
['1020']
['1022']
['10253']
['1026']
['1027']
['10290']
['10303']
['1031']
['1032']
['1034']
['10362']
['1037']
['10379']
['1038']
['10391']
['1040']
['10405']
['1041']
['1042']
['1043']
['1046']
['1047']
['10471']
['10473']
['10487']
['10518']
['10556']
['1057']
['1059']
['1060']
['10617']
['1062']
['1063']
['10643']
['10646']
['1066']
['1067']
['10678']
['1068']
['10683']
['10685']
['1069']
['10690']
['10704']
['1073']
['10765']
['10766']
['10769']
['10790']
['108']
['1080']
['10810']
['10817']
['1082']
['1083']
['10850']
['10868']
['1089']
['10908']
['1092']
['1093']
['1094']
['10940']
['1095']
['1096']
['1099']
['11']
['110']
['11002']
['11006']
['11009']
['1102']
['1103']
['1104']
['11055']
['1106']
['1107']
['11072']
['11091']
['11092']
['11093']
['11095']
['111']
['1110']
['11103']
['11107']
['1111']
['11114']
['1112']
['

Patterns: 
['1'] Support: 0.7165532879818595
['1092'] Support: 0.006802721088435374
['1115'] Support: 0.006802721088435374
['118'] Support: 0.006802721088435374
['120'] Support: 0.006802721088435374
['1215'] Support: 0.006802721088435374
['130'] Support: 0.006802721088435374
['1328'] Support: 0.009070294784580499
['1393'] Support: 0.006802721088435374
['1563'] Support: 0.006802721088435374
['179'] Support: 0.006802721088435374
['18'] Support: 0.006802721088435374
['2'] Support: 0.3854875283446712
['201'] Support: 0.006802721088435374
['239'] Support: 0.006802721088435374
['2405'] Support: 0.006802721088435374
['2406'] Support: 0.006802721088435374
['247'] Support: 0.006802721088435374
['255'] Support: 0.006802721088435374
['2616'] Support: 0.006802721088435374
['283'] Support: 0.006802721088435374
['290'] Support: 0.006802721088435374
['3'] Support: 0.7188208616780045
['3045'] Support: 0.009070294784580499
['314'] Support: 0.006802721088435374
['333'] Support: 0.006802721088435374
['37

Patterns: 
['1', '1092'] Support: 0.006802721088435374
['1', '118'] Support: 0.006802721088435374
['1', '1328'] Support: 0.009070294784580499
['1', '1393'] Support: 0.006802721088435374
['1', '1563'] Support: 0.006802721088435374
['1', '18'] Support: 0.006802721088435374
['1', '2'] Support: 0.10430839002267574
['1', '239'] Support: 0.006802721088435374
['1', '2406'] Support: 0.006802721088435374
['1', '255'] Support: 0.006802721088435374
['1', '290'] Support: 0.006802721088435374
['1', '3'] Support: 0.48072562358276644
['1', '3045'] Support: 0.006802721088435374
['1', '314'] Support: 0.006802721088435374
['1', '333'] Support: 0.006802721088435374
['1', '375'] Support: 0.006802721088435374
['1', '3838'] Support: 0.006802721088435374
['1', '395'] Support: 0.009070294784580499
['1', '397'] Support: 0.006802721088435374
['1', '402'] Support: 0.006802721088435374
['1', '405'] Support: 0.006802721088435374
['1', '436'] Support: 0.006802721088435374
['1', '483'] Support: 0.006802721088435374
